<a href="https://colab.research.google.com/github/JaiVR/nir_chemometrics/blob/main/chemometrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as ex
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

These dataset was collected from the website of radboud university http://webchem.science.ru.nl/chemometrics-i/data-sets/ on 15/12/2016
wines.txt and cultivers.txt contains the data of 177 Italian wines characterized by 13 variables
iris.txt, irisdata.txt and iris2.txt contains the data table of measurement od three types of irises (iris2 with outliers)
soil.txt and spectra.txt has the spectra and physical characteristics of 54 soil samples.
nir_gasoline.txt and octane.txt contains gasoline nir sprectra and octane content


In [2]:
cultivars = pd.read_csv('cultivars.csv')
iris = pd.read_csv('iris.csv')
iris2 = pd.read_csv('iris2.csv')
irisclass = pd.read_csv('irisclass.csv')
soil = pd.read_csv('soil.csv')
spectra = pd.read_csv('spectra.csv')
gasoline = pd.read_csv('nir_gasoline.csv')
octane = pd.read_csv('octane.csv')
wines = pd.read_csv('wines.csv')

In [3]:
column_names_wine = ["Alcohol", "Malic_Acid", "Ash", "Alcalinity_of_Ash", "Magnesium",
                     "Total_Phenols", "Flavanoids", "Nonflavanoid_Phenols", "Proanthocyanins",
                     "Color_Intensity", "Hue", "OD280/OD315", "Proline"]
wines = pd.read_csv('wines.csv', header=None, names=column_names_wine)

cultivers = pd.read_csv('cultivars.csv', header=None, names=["Cultivar"])

column_names_iris = ["Sepal_Length", "Sepal_Width", "Petal_Length", "Petal_Width", "Species"]
iris = pd.read_csv('iris.csv', header=None, names=column_names_iris)
iris2 = pd.read_csv('iris2.csv', header=None, names=column_names_iris)
irisclass = pd.read_csv('irisclass.csv', header=None, names=["Species"])

column_names_soil = [f"property{i+1}" for i in range(300)]  # replace with actual property names
soil = pd.read_csv('soil.csv', header=None, names=column_names_soil).dropna(axis=1, how='all')

column_names_gasoline = [f"wavelength{i}" for i in range(gasoline.shape[1])]
gasoline = pd.read_csv('nir_gasoline.csv', header=None, names=column_names_gasoline)
octane = pd.read_csv('octane.csv', header=None, names=["Octane"])

column_names_spectra = [f"Wavelength_{i+1}" for i in range(spectra.shape[1])]
spectra = pd.read_csv('spectra.csv', header=None, names=column_names_spectra).dropna(axis=1, how='all')
